# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f324c664130>
├── 'a' --> tensor([[ 0.4513, -0.9724, -0.5296],
│                   [ 0.3514, -0.5366,  0.6840]])
└── 'x' --> <FastTreeValue 0x7f324c664070>
    └── 'c' --> tensor([[-1.1569,  0.1718, -0.6007, -0.2880],
                        [ 1.3647, -0.0194,  0.5076, -0.2821],
                        [ 0.8438,  0.7964,  0.2256,  1.6040]])

In [4]:
t.a

tensor([[ 0.4513, -0.9724, -0.5296],
        [ 0.3514, -0.5366,  0.6840]])

In [5]:
%timeit t.a

59.1 ns ± 0.056 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f324c664130>
├── 'a' --> tensor([[ 0.2868,  2.0820, -0.7874],
│                   [ 2.7329,  0.8582,  0.7886]])
└── 'x' --> <FastTreeValue 0x7f324c664070>
    └── 'c' --> tensor([[-1.1569,  0.1718, -0.6007, -0.2880],
                        [ 1.3647, -0.0194,  0.5076, -0.2821],
                        [ 0.8438,  0.7964,  0.2256,  1.6040]])

In [7]:
%timeit t.a = new_value

58.2 ns ± 0.0295 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.4513, -0.9724, -0.5296],
               [ 0.3514, -0.5366,  0.6840]]),
    x: Batch(
           c: tensor([[-1.1569,  0.1718, -0.6007, -0.2880],
                      [ 1.3647, -0.0194,  0.5076, -0.2821],
                      [ 0.8438,  0.7964,  0.2256,  1.6040]]),
       ),
)

In [10]:
b.a

tensor([[ 0.4513, -0.9724, -0.5296],
        [ 0.3514, -0.5366,  0.6840]])

In [11]:
%timeit b.a

55.1 ns ± 0.0737 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.5744,  0.2339, -0.9821],
               [-2.3618,  1.0731,  0.5992]]),
    x: Batch(
           c: tensor([[-1.1569,  0.1718, -0.6007, -0.2880],
                      [ 1.3647, -0.0194,  0.5076, -0.2821],
                      [ 0.8438,  0.7964,  0.2256,  1.6040]]),
       ),
)

In [13]:
%timeit b.a = new_value

484 ns ± 0.0905 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

849 ns ± 0.259 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.7 µs ± 13.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

136 µs ± 259 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

141 µs ± 3.29 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f31a238efa0>
├── 'a' --> tensor([[[ 0.4513, -0.9724, -0.5296],
│                    [ 0.3514, -0.5366,  0.6840]],
│           
│                   [[ 0.4513, -0.9724, -0.5296],
│                    [ 0.3514, -0.5366,  0.6840]],
│           
│                   [[ 0.4513, -0.9724, -0.5296],
│                    [ 0.3514, -0.5366,  0.6840]],
│           
│                   [[ 0.4513, -0.9724, -0.5296],
│                    [ 0.3514, -0.5366,  0.6840]],
│           
│                   [[ 0.4513, -0.9724, -0.5296],
│                    [ 0.3514, -0.5366,  0.6840]],
│           
│                   [[ 0.4513, -0.9724, -0.5296],
│                    [ 0.3514, -0.5366,  0.6840]],
│           
│                   [[ 0.4513, -0.9724, -0.5296],
│                    [ 0.3514, -0.5366,  0.6840]],
│           
│                   [[ 0.4513, -0.9724, -0.5296],
│                    [ 0.3514, -0.5366,  0.6840]]])
└── 'x' --> <FastTreeValue 0x7f324f6d5fa0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.5 µs ± 101 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f324c67a760>
├── 'a' --> tensor([[ 0.4513, -0.9724, -0.5296],
│                   [ 0.3514, -0.5366,  0.6840],
│                   [ 0.4513, -0.9724, -0.5296],
│                   [ 0.3514, -0.5366,  0.6840],
│                   [ 0.4513, -0.9724, -0.5296],
│                   [ 0.3514, -0.5366,  0.6840],
│                   [ 0.4513, -0.9724, -0.5296],
│                   [ 0.3514, -0.5366,  0.6840],
│                   [ 0.4513, -0.9724, -0.5296],
│                   [ 0.3514, -0.5366,  0.6840],
│                   [ 0.4513, -0.9724, -0.5296],
│                   [ 0.3514, -0.5366,  0.6840],
│                   [ 0.4513, -0.9724, -0.5296],
│                   [ 0.3514, -0.5366,  0.6840],
│                   [ 0.4513, -0.9724, -0.5296],
│                   [ 0.3514, -0.5366,  0.6840]])
└── 'x' --> <FastTreeValue 0x7f31a8fa5190>
    └── 'c' --> tensor([[-1.1569,  0.1718, -0.6007, -0.2880],
                        [ 1.3647, -0.0194,  0.5076, -0.2821],
                 

In [23]:
%timeit t_cat(trees)

30.9 µs ± 121 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57.4 µs ± 105 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.4513, -0.9724, -0.5296],
                [ 0.3514, -0.5366,  0.6840]],
       
               [[ 0.4513, -0.9724, -0.5296],
                [ 0.3514, -0.5366,  0.6840]],
       
               [[ 0.4513, -0.9724, -0.5296],
                [ 0.3514, -0.5366,  0.6840]],
       
               [[ 0.4513, -0.9724, -0.5296],
                [ 0.3514, -0.5366,  0.6840]],
       
               [[ 0.4513, -0.9724, -0.5296],
                [ 0.3514, -0.5366,  0.6840]],
       
               [[ 0.4513, -0.9724, -0.5296],
                [ 0.3514, -0.5366,  0.6840]],
       
               [[ 0.4513, -0.9724, -0.5296],
                [ 0.3514, -0.5366,  0.6840]],
       
               [[ 0.4513, -0.9724, -0.5296],
                [ 0.3514, -0.5366,  0.6840]]]),
    x: Batch(
           c: tensor([[[-1.1569,  0.1718, -0.6007, -0.2880],
                       [ 1.3647, -0.0194,  0.5076, -0.2821],
                       [ 0.8438,  0.7964,  0.2256,  1.6040]],
         

In [26]:
%timeit Batch.stack(batches)

79.4 µs ± 164 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.4513, -0.9724, -0.5296],
               [ 0.3514, -0.5366,  0.6840],
               [ 0.4513, -0.9724, -0.5296],
               [ 0.3514, -0.5366,  0.6840],
               [ 0.4513, -0.9724, -0.5296],
               [ 0.3514, -0.5366,  0.6840],
               [ 0.4513, -0.9724, -0.5296],
               [ 0.3514, -0.5366,  0.6840],
               [ 0.4513, -0.9724, -0.5296],
               [ 0.3514, -0.5366,  0.6840],
               [ 0.4513, -0.9724, -0.5296],
               [ 0.3514, -0.5366,  0.6840],
               [ 0.4513, -0.9724, -0.5296],
               [ 0.3514, -0.5366,  0.6840],
               [ 0.4513, -0.9724, -0.5296],
               [ 0.3514, -0.5366,  0.6840]]),
    x: Batch(
           c: tensor([[-1.1569,  0.1718, -0.6007, -0.2880],
                      [ 1.3647, -0.0194,  0.5076, -0.2821],
                      [ 0.8438,  0.7964,  0.2256,  1.6040],
                      [-1.1569,  0.1718, -0.6007, -0.2880],
                      [ 1.3647, -

In [28]:
%timeit Batch.cat(batches)

144 µs ± 153 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

310 µs ± 907 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
